This is a version of the `solution-base` notebook that allows for iterating automatically through multiple settings based on the values in an Excel spreadsheet.

It is recommended that you fully understand `solution-base` before implementing this automated version.

# Imports

In [1]:
import cs7641assn4 as a4
import numpy as np
import pandas as pd
import warnings
import xlrd

pd.set_option('display.max_columns', 35)
# pd.reset_option("display.max_columns")

In [2]:
# Default settings, uncomment to just use these
# settings = {'rH': [-1],
#  'rG': [1],
#  'rF': [-0.2],
#  'size': [4],
#  'p': [0.8],
#  'is_slippery': [False],
#  'render_initial': [True],
#  'epsilon': [1e-08],
#  'gamma': [0.8],
#  'max_iter': [10000],
#  'qepsilon': [0.1],
#  'lr': [0.8],
#  'qgamma': [0.95],
#  'episodes': [10000],
#  'initial': [1],
#  'decay': [True],
#  'report': [True],
#  'display_print': [True]}
# Import settings, uncomment to read settings from an excel spreadsheet
settings = pd.read_excel('settings.xlsx').to_dict()

# Determine the number of runs
n_settings = len(settings['rH'])


In [3]:
for n in range(n_settings):
    # Load settings
    rH = settings['rH'][n] # -1 #-5 # reward for H(ole)
    rG = settings['rG'][n] #1 # 10 # reward for G(oal)
    rF = settings['rF'][n] #-0.2# reward includes S(tart) and F(rozen)
    size = settings['size'][n] #4 # height and width of square gridworld, [4, 8, 16] are included in cs7641assn4.py 
    p = settings['p'][n] #0.8 # if generating a random map probability that a grid will be F(rozen)
    map_name = 'x'.join([str(size)]*2) # None, if you want a random map
    desc = a4.MAPS[map_name] # None, if you want a random map
    is_slippery = settings['is_slippery'][n] #False
    render_initial = settings['render_initial'][n] # True

    epsilon = settings['epsilon'][n] #1e-8 # convergence threshold for policy/value iteration
    gamma = settings['gamma'][n] #0.8 # discount parameter for past policy/value iterations
    max_iter = settings['max_iter'][n] #10000 # maximum iterations for slowly converging policy/value iteration 

    # Qlearning(env, rH=0, rG=1, rF=0, qepsilon=0.1, lr=0.8, gamma=0.95, episodes=10000)
    qepsilon = settings['qepsilon'][n] #0.1 # epsilon value for the Q-learning epsilon greedy strategy
    lr = settings['lr'][n] #0.8 # Q-learning rate
    qgamma = settings['qgamma'][n] #0.95 # Q-Learning discount factor
    episodes = settings['episodes'][n] #10000 # number of Q-learning episodes
    initial = settings['initial'][n] #0 # value to initialize the Q grid
    decay = settings['decay'][n] #True

    # Printing options
    report = settings['report'][n] #True # For cs7641assn4.py policy and value iteration functions
    display_print = settings['display_print'][n] #True # For this script

    # Create Environment
    env = a4.getEnv(env_id='hw4-FrozenLake-v0', rH=rH, rG=rG, rF=rF, 
                    desc=desc,  
                    is_slippery=is_slippery, render_initial=True)

    # Store a representation of the map
    env_desc = env.desc.astype('<U8')

    # Store a representation of the state rewards
    env_rs = a4.getStateReward(env)

    if display_print:
        # Display reward at each state
        print('\n--Reward Values at Each State--')
        a4.matprint(a4.print_value(env_rs, width=size, height=size))
        
    ## Policy Iteration
    print('\n--Policy Iteration TimeIt--')
    pi_time = %timeit -o a4.policy_iteration(env, epsilon, gamma, max_iter, report=False)
    
    pi_V, pi_policy, pi_epochs = a4.policy_iteration(env, epsilon, gamma, max_iter, report=report)

    pi_policy_arrows = a4.print_policy(pi_policy, width=size, height=size)

    if display_print:
        # Display values
        print('\n--Policy Iteration Values in grid order--')
        a4.matprint(a4.print_value(pi_V, width=size, height=size))

        # Display policy
        print('\n--Policy Iteration Policy Matrix--')
        a4.matprint(pi_policy_arrows)
        
    ## Value Iteration
    print('\n--Value Iteration TimeIt--')
    vi_time = %timeit -o a4.valueIteration(env, epsilon, gamma, max_iter, report=False)
    
    vi_V, vi_epochs = a4.valueIteration(env, epsilon, gamma, max_iter, report=report)

    vi_policy = a4.value_to_policy(env, V=vi_V, gamma=gamma)

    vi_policy_arrows = a4.print_policy(vi_policy, width=size, height=size)

    if display_print:
        # display value function:
        print('\n--Value Iteration Values in grid order--')
        a4.matprint(a4.print_value(vi_V, width=size, height=size))
        
        # display policy
        print('\n--Value Iteration Policy Matrix--')
        a4.matprint(vi_policy_arrows)
        
    ## Q-Learning
    print('\n--Q-Learning TimeIt--')
    Q_time = %timeit -o a4.Qlearning(env, qepsilon, lr, qgamma, episodes, initial, decay, report=False)
        
    Q, Q_epochs = a4.Qlearning(env, qepsilon, lr, qgamma, episodes, initial, decay, report)

    maxQ = np.max(Q,axis=1)

    Q_policy = a4.Q_to_policy(Q)

    Q_policy_arrows = a4.print_policy(Q_policy, width=size, height=size)

    if display_print: 
        print('--Q with all options--')
        a4.matprint(Q)
        print('\n--argmax(Q) in grid order--')
        a4.matprint(a4.print_value(maxQ, width=size, height=size))
        print('\n--Q-Learning Policy Matrix--')
        a4.matprint(Q_policy_arrows)
        
    ## Save results to DataFrame
    results = pd.DataFrame({'rH': [rH], 
                        'rG': [rG], 
                        'rF': [rF], 
                        'size': [size], 
                        'p': [p], 
                        'desc': [desc], 
                        'map_name': [map_name],                        
                        'is_slippery': [is_slippery],
                        'epsilon': [epsilon],
                        'gamma': [gamma], 
                        'max_iter': [max_iter], 
                        'qepsilon': [qepsilon], 
                        'lr': [lr], 
                        'qgamma': [qgamma], 
                        'episodes': [episodes], 
                        'initial': [initial],
                        'env_desc': [env_desc],
                        'env_rs': [env_rs],
                        'pi_time': [pi_time.average],
                        'pi_V': [pi_V],
                        'pi_epochs': [pi_epochs],
                        'pi_policy': [pi_policy],
                        'pi_policy_arrows': [pi_policy_arrows],
                        'vi_time': [vi_time.average],
                        'vi_V': [vi_V],
                        'vi_epochs': [vi_epochs],
                        'vi_policy': [vi_policy],
                        'vi_policy_arrows': [vi_policy_arrows],
                        'Q_time': [Q_time.average],
                        'Q': [Q],
                        'Q_epochs': [Q_epochs],
                        'Q_V': [maxQ],
                        'Q_policy': [Q_policy],
                        'Q_policy_arrows': [Q_policy_arrows]})
    
    if display_print: 
        display(results)
        
    ## Save results to disk
    warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
    try:
        dataset = pd.read_hdf('data.h5', key='dataset', mode='a')
    except FileNotFoundError:
        results.to_hdf('data.h5', key='dataset', mode='a')
    else:
        dataset.append(
            other=results, 
            ignore_index=True,
            sort=False
            ).to_hdf(
            path_or_buf='data.h5', 
            key='dataset', 
            mode='a')
        
    if display_print:
        pd.read_hdf('data.h5', key='dataset', mode='a')
    
    
print('Complete!')   

--Board--

S  F  F  F
F  H  F  H
F  F  F  H
H  F  F  G

--Actions for Position to the Left of the Goal--
{0: [(1.0, 13, -0.2, False)],
 1: [(1.0, 14, -0.2, False)],
 2: [(1.0, 15, 1, True)],
 3: [(1.0, 10, -0.2, False)]}

--Reward Values at Each State--
-0.2  -0.2  -0.2  -0.2  
-0.2    -1  -0.2    -1  
-0.2  -0.2  -0.2    -1  
  -1  -0.2  -0.2     1  

--Policy Iteration TimeIt--
980 µs ± 59.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Policy iteration converged after  7 epochs

--Policy Iteration Values in grid order--
0.9661  1.4576  2.072  1.4576  
1.4576      -5   2.84      -5  
 2.072    2.84    3.8      -5  
    -5     3.8      5       5  

--Policy Iteration Policy Matrix--
↓  →  ↓  ←  
↓  ←  ↓  ←  
→  ↓  ↓  ←  
←  →  →  ←  

--Value Iteration TimeIt--
7.69 ms ± 625 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Value iteration converged after  91 epochs

--Value Iteration Values in grid order--
0.9661  1.4576  2.072  1.4576  
1.4576      -5   2.84      

,rH,rG,rF,size,p,desc,map_name,is_slippery,epsilon,gamma,max_iter,qepsilon,lr,qgamma,episodes,initial,env_desc,env_rs,pi_time,pi_V,pi_epochs,pi_policy,pi_policy_arrows,vi_time,vi_V,vi_epochs,vi_policy,vi_policy_arrows,Q_time,Q,Q_epochs,Q_V,Q_policy,Q_policy_arrows
0,-1,1,-0.2,4,0.8,"[SFFF, FHFH, FFFH, HFFG]",4x4,False,1.000000e-08,0.8,10000,0.1,0.8,0.95,10000,1,"[[S, F, F, F], [F, H, F, H], [F, F, F, H], [H,...","[-0.2, -0.2, -0.2, -0.2, -0.2, -1.0, -0.2, -1....",0.00098,"[0.9660800000000008, 1.457600000000001, 2.0720...",7,"[1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]","[[↓, →, ↓, ←], [↓, ←, ↓, ←], [→, ↓, ↓, ←], [←,...",0.007688,"[0.9660799905143127, 1.4575999905143127, 2.071...",91,"[1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]","[[↓, →, ↓, ←], [↓, ←, ↓, ←], [→, ↓, ↓, ←], [←,...",0.011498,"[[0.3730120111912959, 0.6039952952128212, 0.13...",35,"[0.6039952952128212, 0.4284927139839999, 0.793...","[1, 2, 1, 2, 1, 0, 1, 0, 2, 2, 1, 0, 0, 2, 2, 0]","[[↓, →, ↓, →], [↓, ←, ↓, ←], [→, →, ↓, ←], [←,..."


--Board--

S  F  F  F  F  F  F  F
F  F  F  F  F  F  F  F
F  F  F  H  F  F  F  F
F  F  F  F  F  H  F  F
F  F  F  H  F  F  F  F
F  H  H  F  F  F  H  F
F  H  F  F  H  F  H  F
F  F  F  H  F  F  F  G

--Actions for Position to the Left of the Goal--
{0: [(1.0, 61, -0.2, False)],
 1: [(1.0, 62, -0.2, False)],
 2: [(1.0, 63, 1, True)],
 3: [(1.0, 54, -1, True)]}

--Reward Values at Each State--
-0.2  -0.2  -0.2  -0.2  -0.2  -0.2  -0.2  -0.2  
-0.2  -0.2  -0.2  -0.2  -0.2  -0.2  -0.2  -0.2  
-0.2  -0.2  -0.2    -1  -0.2  -0.2  -0.2  -0.2  
-0.2  -0.2  -0.2  -0.2  -0.2    -1  -0.2  -0.2  
-0.2  -0.2  -0.2    -1  -0.2  -0.2  -0.2  -0.2  
-0.2    -1    -1  -0.2  -0.2  -0.2    -1  -0.2  
-0.2    -1  -0.2  -0.2    -1  -0.2    -1  -0.2  
-0.2  -0.2  -0.2    -1  -0.2  -0.2  -0.2     1  

--Policy Iteration TimeIt--
11.7 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
Policy iteration converged after  13 epochs

--Policy Iteration Values in grid order--
-0.6701  -0.5877  -0.4846  -0.

,rH,rG,rF,size,p,desc,map_name,is_slippery,epsilon,gamma,max_iter,qepsilon,lr,qgamma,episodes,initial,env_desc,env_rs,pi_time,pi_V,pi_epochs,pi_policy,pi_policy_arrows,vi_time,vi_V,vi_epochs,vi_policy,vi_policy_arrows,Q_time,Q,Q_epochs,Q_V,Q_policy,Q_policy_arrows
0,-1,1,-0.2,8,0.8,"[SFFFFFFF, FFFFFFFF, FFFHFFFF, FFFFFHFF, FFFHF...",8x8,False,1.000000e-08,0.8,10000,0.1,0.8,0.95,10000,1,"[[S, F, F, F, F, F, F, F], [F, F, F, F, F, F, ...","[-0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0....",0.011709,"[-0.6701465116672001, -0.5876831395840001, -0....",13,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, ...","[[↓, ↓, ↓, ↓, ↓, ↓, ↓, ↓], [↓, →, ↓, →, ↓, ↓, ...",0.035635,"[-0.6701465165238721, -0.587683144440672, -0.4...",94,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, ...","[[↓, ↓, ↓, ↓, ↓, ↓, ↓, ↓], [↓, ↓, ↓, →, ↓, ↓, ...",0.069466,"[[-1.0983474144849827, -0.9954115242107094, -0...",101,"[-0.9456146044884779, -0.7848574784075173, -0....","[2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, ...","[[→, →, →, →, ↓, ↓, →, ↓], [→, →, →, →, ↓, ↓, ..."


--Board--

S  F  F  F
F  H  F  H
F  F  F  H
H  F  F  G

--Actions for Position to the Left of the Goal--
{0: [(0.3333333333333333, 10, -0.2, False),
     (0.3333333333333333, 13, -0.2, False),
     (0.3333333333333333, 14, -0.2, False)],
 1: [(0.3333333333333333, 13, -0.2, False),
     (0.3333333333333333, 14, -0.2, False),
     (0.3333333333333333, 15, 1, True)],
 2: [(0.3333333333333333, 14, -0.2, False),
     (0.3333333333333333, 15, 1, True),
     (0.3333333333333333, 10, -0.2, False)],
 3: [(0.3333333333333333, 15, 1, True),
     (0.3333333333333333, 10, -0.2, False),
     (0.3333333333333333, 13, -0.2, False)]}

--Reward Values at Each State--
-0.2  -0.2  -0.2  -0.2  
-0.2    -1  -0.2    -1  
-0.2  -0.2  -0.2    -1  
  -1  -0.2  -0.2     1  

--Policy Iteration TimeIt--
831 µs ± 121 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Policy iteration converged after  4 epochs

--Policy Iteration Values in grid order--
-0.9257  -0.9676  -0.9851  -0.9915  
  -0.87       -5  -

...

,rH,rG,rF,size,p,desc,map_name,is_slippery,epsilon,gamma,max_iter,qepsilon,lr,qgamma,episodes,initial,env_desc,env_rs,pi_time,pi_V,pi_epochs,pi_policy,pi_policy_arrows,vi_time,vi_V,vi_epochs,vi_policy,vi_policy_arrows,Q_time,Q,Q_epochs,Q_V,Q_policy,Q_policy_arrows
0,-1,1,-0.2,8,0.8,"[SFFFFFFF, FFFFFFFF, FFFHFFFF, FFFFFHFF, FFFHF...",8x8,True,1.000000e-08,0.8,10000,0.1,0.8,0.95,10000,0,"[[S, F, F, F, F, F, F, F], [F, F, F, F, F, F, ...","[-0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0....",0.004994,"[-0.9992295366480467, -0.9986516891340815, -0....",7,"[3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, ...","[[↑, →, →, →, →, →, →, →], [↑, ↑, ↑, ↑, ↑, ↑, ...",0.029561,"[-0.999229537127132, -0.9986516896132036, -0.9...",91,"[3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, ...","[[↑, →, →, →, →, →, →, →], [↑, ↑, ↑, ↑, ↑, ↑, ...",4.809514,"[[-2.7148852803353805, -2.631316292974166, -2....",10000,"[-2.631316292974166, -2.6238578894993716, -2.6...","[1, 2, 1, 1, 1, 0, 1, 3, 1, 2, 1, 1, 1, 0, 1, ...","[[↓, →, ↓, ↓, ↓, ←, ↓, ↑], [↓, →, ↓, ↓, ↓, ←, ..."


--Board--

S  F  F  F  F  F  H  H  F  F  F  F  F  F  F  F
F  F  F  F  H  F  F  H  F  F  F  F  F  F  H  F
F  H  F  F  F  F  F  H  F  H  F  H  F  F  F  F
H  F  F  H  F  F  F  F  F  F  F  F  F  H  F  H
F  H  F  F  F  F  H  H  F  F  F  H  H  F  H  F
F  H  F  F  H  F  H  F  F  F  F  H  F  F  F  F
F  F  H  F  F  F  F  H  F  F  F  H  H  H  H  F
H  F  H  F  F  F  F  F  F  F  F  F  F  F  F  H
F  F  F  F  F  F  F  F  F  H  F  F  F  F  H  F
F  F  F  F  F  F  H  H  F  H  F  F  F  F  F  H
F  F  H  F  F  F  F  F  F  H  F  F  F  H  F  F
F  F  F  H  H  F  H  F  F  F  F  F  H  F  F  F
F  F  F  F  F  H  F  H  F  F  F  F  H  F  F  F
H  H  H  H  H  F  F  F  H  F  F  H  F  F  F  F
F  H  F  F  H  F  F  F  F  F  F  F  F  F  F  F
F  F  H  F  H  H  F  F  H  F  F  F  F  F  F  G

--Actions for Position to the Left of the Goal--
{0: [(1.0, 253, -0.2, False)],
 1: [(1.0, 254, -0.2, False)],
 2: [(1.0, 255, 1, True)],
 3: [(1.0, 238, -0.2, False)]}

--Reward Values at Each State--
-0.2  -0.2  -0.2  -0.2  -0.2  -0.2

155 ms ± 32.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Q-Learning converged after  223 epochs
--Q with all options--
 -2.24177   -2.14985   -2.14987   -2.24222  
 -2.20384    -2.0525   -2.05907   -2.14959  
 -2.10951   -1.96097   -1.97207   -2.02908  
 -2.01801   -1.87492   -1.87624   -1.93187  
  -1.8988    -1.9344   -1.79689   -1.80488  
 -1.76862   -1.71388    -1.9344   -1.77861  
       -1         -1         -1         -1  
       -1         -1         -1         -1  
    -1.56   -0.99184  -0.901504   -0.99421  
-0.941789   -0.98797  -0.968565   -1.02012  
  -1.0152  -0.951123  -0.958143   -0.99421  
-0.872701   -0.92306  -0.970166  -0.868969  
-0.885975   -1.04368   -0.94503  -0.868969  
-0.945221   -1.01227  -0.994485   -0.99421  
 -1.03764     -1.872   -1.11065   -0.99421  
 -1.09801   -1.20672   -1.11444   -1.11444  
 -2.14161   -2.05249    -2.0525   -2.24082  
 -2.01654      -1.95    -1.9539   -2.09688  
 -2.02418   -1.85925   -1.86628     -1.994  
 -1.85754   -1.

       -1         -1         -1         -1  
       -1         -1         -1         -1  
       -1         -1         -1         -1  
       -1         -1         -1         -1  
       -1         -1         -1         -1  
    -1.56          0          0          0  
    -0.16          0          0          0  
    -0.16          0          0          0  
       -1         -1         -1         -1  
    -1.56  -0.473354  -0.436736  -0.509522  
  -0.4416    -0.4672      -1.56  -0.473139  
       -1         -1         -1         -1  
        0          0          0          0  
        0          0          0          0  
        0          0          0          0  
        0          0          0          0  
        0          0          0          0  
        0          0          0          0  
        0          0          0          0  
        0          0          0          0  
       -1         -1         -1         -1  
    -1.56          0          0          0  
    -0.16 

,rH,rG,rF,size,p,desc,map_name,is_slippery,epsilon,gamma,max_iter,qepsilon,lr,qgamma,episodes,initial,env_desc,env_rs,pi_time,pi_V,pi_epochs,pi_policy,pi_policy_arrows,vi_time,vi_V,vi_epochs,vi_policy,vi_policy_arrows,Q_time,Q,Q_epochs,Q_V,Q_policy,Q_policy_arrows
0,-1,1,-0.2,16,0.8,"[SFFFFFHHFFFFFFFF, FFFFHFFHFFFFFFHF, FHFFFFFHF...",16x16,False,1.000000e-08,0.8,10000,0.1,0.8,0.95,10000,0,"[[S, F, F, F, F, F, H, H, F, F, F, F, F, F, F,...","[-0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -1.0, -1....",0.131494,"[-0.99071544970536, -0.9883943121316999, -0.98...",19,"[1, 1, 1, 1, 2, 1, 0, 0, 2, 1, 1, 0, 0, 0, 0, ...","[[↓, ↓, ↓, ↓, →, ↓, ←, ←, →, ↓, ↓, ←, ←, ←, ←,...",0.122988,"[-0.9907154521919761, -0.9883943146183161, -0....",97,"[1, 1, 1, 1, 2, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, ...","[[↓, ↓, ↓, ↓, →, ↓, ←, ←, ↓, ↓, ↓, ↓, ↓, ↓, ←,...",0.155251,"[[-2.241765079546978, -2.1498522743980164, -2....",223,"[-2.1498522743980164, -2.0524999920506035, -1....","[1, 1, 1, 1, 2, 1, 0, 0, 2, 0, 1, 3, 3, 0, 3, ...","[[↓, ↓, ↓, ↓, →, ↓, ←, ←, →, ←, ↓, ↑, ↑, ←, ↑,..."


# Notes

Default rewards in OpenAI gym Frozen-Lake-v0 are 1 for the G(oal) and 0 for everything else.

Maps are drawn according to the following logic

```
if desc and map_name are None, 
   then a default random map is drawn with 8
        using frozen_lake.generate_random_map(size=8, p=0.8)
elif desc is None and a map_name is given
   then a map_name is either '4x4' or '8x8'
        and is drawn from the dict MAPS in frozen_lake.py
elif desc is given
   then it must be in the form of a list with 
```

Default action probabilities are 1/3 chosen action, 1/3 each for right angles to chosen action, and 0 for reverse of chosen action. This is set with `is_slippery=True`. If `is_slippery=False`, then P=1 for chosen action and 0 for all other actions.

|ACTION|Value|Symbol|
|------|-----|------|
|LEFT  | 0   | ←    |
|DOWN  | 1   | ↓    |
|RIGHT | 2   | →    |
|UP    | 3   | ↑    |

# Sources

- Environment: <https://gym.openai.com/envs/FrozenLake-v0/>
- Code: <https://github.com/Twice22/HandsOnRL>
- Tutorial: <https://twice22.github.io/>